In [3]:
###################################################
# IMPORTANTE: Instalar la base de datos siguiendo #
# las instrucciones del README e indicar en estas #
# variables los direcctorios!!                    #
TRAIN_REAL_DATA_PATH = "D:\Datos\TrainReal"
TRAIN_SIM_DATA_PATH  = "D:\Datos\TrainSimulation"
TEST_DATA_PATH       = "D:\Datos\TestReal"
###################################################

OUTPUT_MODEL_PATH = "model"
OUTPUT_INFERENCE_PATH = "output"

## Inicialización (COLAB)

In [ ]:
# #####################################################
# # Si se ejecuta desde colab, descomentar esta celda #
# #####################################################

# ## Para descargar el proyecto e importar todas las dependencias
# !git clone https://github.com/jesusBV20/UAV_IR_Surveillance.git
# !mv UAV_IR_Surveillance/* . 
# !rm -r UAV_IR_Surveillance

Cloning into 'UAV_IR_Surveillance'...
remote: Enumerating objects: 198, done.
remote: Counting objects: 100% (198/198), done.
remote: Compressing objects: 100% (153/153), done.
remote: Total 198 (delta 88), reused 140 (delta 40), pack-reused 0
Receiving objects: 100% (198/198), 6.46 MiB | 15.59 MiB/s, done.
Resolving deltas: 100% (88/88), done.


In [ ]:
# #####################################################
# # Si se ejecuta desde colab, descomentar esta celda #
# #####################################################

# ## ¿Qué CPU nos ha asignado google?
# !cat /proc/cpuinfo | grep "model name"

# ## ¿Qué GPU nos ha asignado google?
# !nvidia-smi

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0xffffffff
cpu MHz		: 2199.998
cache size	: 56320 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa mmio_stale_data retbleed
bogomips	: 4399.99
clflush size	: 64
cache_alignment	: 64
addres

In [ ]:
# #####################################################
# # Si se ejecuta desde colab, descomentar esta celda #
# #####################################################

# ## Para descargar y descomprimir los datos de forma automática --
# from IPython.display import clear_output

# # TrainReal
# !wget "https://storage.googleapis.com/public-datasets-lila/conservationdrones/v01/conservation_drones_train_real.zip"
# !unzip conservation_drones_train_real.zip
# clear_output()

# # TestReal
# !wget "https://storage.googleapis.com/public-datasets-lila/conservationdrones/v01/conservation_drones_test_real.zip"
# !unzip conservation_drones_test_real.zip
# clear_output()

# # print("Done!")

## Cargamos librerías

In [4]:
import os
import numpy as np
import pandas as pd
import matplotlib.pylab as plt

import torch
import torchvision.transforms as T
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from UAVIR_tools import BIRDSAIDataset, imgBoxes, createDir

# Dimensiones de las figuras
FIGSIZE = [16, 9]
#RES = 1920 # Full HD
RES = 2560 # 2k

# Si se encuentra disponible, seleccionamos GPU como dispositivo para entrenar
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("Device: {}\n".format(device))

"""
Función que toma Faster-RCNN y genera un nuevo modelo
aplicando un fine-tunning en el clasificador, adaptándolo al número
de clases, que debe de ser indicado en 'num_classes'. 
"""
def get_tunned_model(num_classes):
    # Cargamos el modelo
    model = fasterrcnn_resnet50_fpn()

    # Reemplazamos el clasificador de la red
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

"""
Función que nos retorna las transformaciones necesarias para codificar las
imágenes (en formato PIL), de tal forma que sean interpretables por la red.
"""
def get_transform():
    transforms = []
    transforms.append(T.PILToTensor())
    transforms.append(T.ConvertImageDtype(torch.float))
    return T.Compose(transforms) # __call__(<PIL image>)

"""
Función para calcular el tamaño de un modelo. Es bastante simple,
únicamente se fija en lo que ocupa el archivo '.pt'.
"""
def print_model_size(mdl):
    torch.save(mdl.state_dict(), "tmp.pt")
    print("%.2f MB" %(os.path.getsize("tmp.pt")/1e6))
    os.remove('tmp.pt')

# Transformaciones necesarias para que la entrada pueda
# ser interpretada por el modelo
transforms = get_transform()

# Creamos el directorio donde vamos a guardar la salida
createDir(OUTPUT_INFERENCE_PATH)



Device: cuda

¡El directorio 'output' ya existe!


## **Cuantizamos el modelo**

In [5]:
################################
# Cargamos el modelo entrenado #
################################
num_classes = 3
model = get_tunned_model(num_classes)

model.to(torch.device('cpu'))
model.load_state_dict(torch.load(os.path.join(OUTPUT_MODEL_PATH, "model_AUVIR_v0.3.pt")))

<All keys matched successfully>

### Cuantizamos

In [6]:
# Inicualizamos el QuantWrappet para añadir funciones de
# cuantización y de-cuantización
quantized_model = torch.quantization.QuantWrapper(model)

# Cuantizamos utilizando un backend concreto
backend = "fbgemm"
model.qconfig = torch.quantization.get_default_qconfig(backend)
torch.backends.quantized.engine = backend

model_static_quantized = torch.quantization.quantize_dynamic(quantized_model)

# Mostramos el tamaño de ambos modelos, el base y el cuantizado
print_model_size(model)
print_model_size(model_static_quantized)

c:\Users\rema0\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\ao\quantization\observer.py:214: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


165.72 MB
124.00 MB


## Generamos un clip infiriendo con el modelo cuantizado

In [5]:
from UAVIR_tools.gen_movie import genMovie

###########################
# Seleccionamos el vídeo  #
movie_id = 15                                     
data_path = TEST_DATA_PATH

score_th = 0.2
init_frame = 1000
end_frame = 1400
###########################

# Cargamos el dataset
dataset = BIRDSAIDataset(data_path, transforms)

# Generamos el vídeo ---
model_static_quantized.to(torch.device('cpu'))
for i in [15]:
  genMovie(i, dataset, model_static_quantized, score_th, output_imgs_path="tmp2",
           init_frame = init_frame, end_frame = end_frame, device="cpu")

Device: cpu

Generando imágenes... 


  0%|          | 0/401 [00:00<?, ?it/s]

## Comprobamos tiempos de inferencia

In [8]:
# Cargamos el dataset
data_path = TEST_DATA_PATH
dataset = BIRDSAIDataset(data_path, transforms)

In [9]:
%%time 
# Modelo sin cuantizar en GPU

model.to(device)
model_eval = model.eval()

for i in range(20):
    output = model_eval([dataset[i][0].to(device)])

CPU times: total: 23 s
Wall time: 14.2 s


In [ ]:
%%time 
# Modelo sin cuantizar en CPU

model.to("cpu")
model_eval = model.eval()

for i in range(20):
    output = model_eval([dataset[i][0].to("cpu")])

In [ ]:
%%time 
# Modelo cuantizado (únicamente soportado en CPU)

model_static_quantized.to("cpu")
model_eval = model_static_quantized.eval()

for i in range(20):
    output = model_eval([dataset[i][0].to("cpu")])